In [1]:
import keras
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D
import numpy as np

def normalize(x):
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))
    x = x - vgg_mean
    return x


Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(Lambda(normalize, input_shape=(224, 224, 3), output_shape=(224, 224, 3)))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

In [3]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 224, 224, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 224, 224, 64)  1792        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 224, 224, 64)  36928       convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 112, 112, 64)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [4]:
# download pretrained model weights from github
# https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5
model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

In [5]:
from keras.preprocessing import image
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [6]:
path = 'dogscats/'
batch_size = 64
train_batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
# now remove the last layer which was trained to do 1000 class
model.pop()

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 224, 224, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 224, 224, 64)  1792        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 224, 224, 64)  36928       convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 112, 112, 64)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [9]:
# lets also set all previous layer to non trainable
for layer in model.layers:
    layer.trainable = False

In [10]:
# we have two classes, CAT and DOG
classes = list(iter(train_batches.class_indices))
num_class = len(classes)
model.add(Dense(num_class, activation='softmax'))

In [11]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 224, 224, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 224, 224, 64)  1792        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 224, 224, 64)  36928       convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 112, 112, 64)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [12]:
from keras.optimizers import Adam
# compile the model with Adam optimizer
learning_rate = 0.001
model.compile(optimizer=Adam(lr=learning_rate),
                loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample, nb_epoch=1,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
23000/23000 [==============================] - 690s - loss: 0.1708 - acc: 0.9547 - val_loss: 0.0819 - val_acc: 0.9750

In [14]:
model.save('cats_vs_dogs_epoch1.h5')

In [30]:
gen=image.ImageDataGenerator()
test_batch = gen.flow_from_directory(path+'test1', target_size=(224,224), class_mode=None,
                       shuffle=False, batch_size=1)

Found 12500 images belonging to 1 classes.


In [31]:
model.predict_generator(test_batch, test_batch.nb_sample)

array([[  9.26223720e-09,   1.00000000e+00],
       [  1.00000000e+00,   2.93746849e-09],
       [  1.58397462e-09,   1.00000000e+00],
       ..., 
       [  1.00000000e+00,   5.50946832e-12],
       [  8.75527914e-11,   1.00000000e+00],
       [  7.98496202e-08,   9.99999881e-01]], dtype=float32)

In [32]:
predictions = _

In [33]:
len(predictions)

12500

In [34]:
predictions.shape

(12500, 2)

In [35]:
predictions[:5]

array([[  9.26223720e-09,   1.00000000e+00],
       [  1.00000000e+00,   2.93746849e-09],
       [  1.58397462e-09,   1.00000000e+00],
       [  1.52090411e-06,   9.99998450e-01],
       [  9.85460318e-08,   9.99999881e-01]], dtype=float32)

In [37]:
test_batch.filenames[:5]

['unknown/160.jpg',
 'unknown/1138.jpg',
 'unknown/1470.jpg',
 'unknown/7901.jpg',
 'unknown/5291.jpg']

In [38]:
classes

['cats', 'dogs']

In [42]:
filenames = test_batch.filenames

In [49]:
test = ''
for i in range(len(filenames)):
    image_id = filenames[i].split('/')[1].split('.jpg')[0]
    cat_prob = int(predictions[i][0])
    is_cat = 1 - cat_prob
    test = test + str(image_id) + ',' + str(is_cat) + '\n'

In [50]:
f = open('dogs_cats_epoch1_submit.csv', 'w')
f.write(test)
f.close()

In [ ]:
# got 1100 by truncating to 0.98 for dog and cats 0.02